# AI Community @ Семинар  №3
## Pandas

**Pandas** - это библиотека Python, предоставляющая широкие возможности для анализа данных. С ее помощью очень удобно загружать, обрабатывать и анализировать табличные данные с помощью SQL-подобных запросов.  
В связке с библиотеками Matplotlib и Seaborn появляется возможность удобного визуального анализа табличных данных. Мы посмотрим на них позже.

In [2]:
# Отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# Знакомые нам вещи
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Стандартное сокращение для pandas - pd
import pandas as pd

Основными структурами данных в **Pandas** являются классы **Series** и **DataFrame**.  
Первый из них представляет собой одномерный индексированный массив данных некоторого фиксированного типа. Мы можем думать о Series как о векторе из прошлого занятия.  
Второй - это двухмерная структура данных (матрица), представляющая собой таблицу, каждый столбец которой содержит данные одного типа. Можно представлять её как словарь объектов типа Series.  
Структура DataFrame отлично подходит для представления реальных данных: строки соответствуют признаковым описаниям отдельных объектов, а столбцы соответствуют признакам.

Для начала рассмотрим простые примеры создания таких объектов и возможных операций над ними.

### Series

Создание объекта Series из 5 элементов, индексированных словами:

In [4]:
salaries = pd.Series([80000, 65000, 20000, 75000], 
              index = ['Андрей', 'Владимир', 'Чарльз', 'Анна']) 
print(salaries)        

Андрей      80000
Владимир    65000
Чарльз      20000
Анна        75000
dtype: int64


Посмотрим на среднюю зарплату.  
Функции numpy принимают pd.Series, так как для него они выглядят как np.array:

In [6]:
np.mean(salaries)

60000.0

Можно сделать то же самое, обратившись к самому объекту pd.Series:

In [7]:
salaries.mean()

60000.0

Посмотрим на людей, чья зарплата выше средней:

In [ ]:
salaries[salaries > salaries.mean()]

Мы можем обращаться к элементам pd.Series как `salaries['Name']` или `salaries.Name`. Например:

In [11]:
salaries.Андрей, salaries['Андрей']

(80000, 80000)

Можно добавлять новые элементы, обращаясь к несуществующему элементу:

In [12]:
salaries['Кот'] = 300000
salaries

Андрей       80000
Владимир     65000
Чарльз       20000
Анна         75000
Кот         300000
dtype: int64

Индексом может быть строка, состоящая из нескольких слов.  
Также, значением в pd.Series может быть `None`, точнее, его аналог в `numpy - np.nan` (not a number):  

In [14]:
salaries['Сергей Бабочка'] = np.nan
salaries

Андрей             80000.0
Владимир           65000.0
Чарльз             20000.0
Анна               75000.0
Кот               300000.0
Сергей Бабочка         NaN
dtype: float64

В данных часто бывают пропуски, поэтому вы часто будете видеть `np.nan`.  
Важно уметь находить их и обрабатывать.  
Получим битовую маску для пропущенных значений: 

In [15]:
salaries.isnull()

Андрей            False
Владимир          False
Чарльз            False
Анна              False
Кот               False
Сергей Бабочка     True
dtype: bool

In [16]:
salaries[salaries.isnull()]

Сергей Бабочка   NaN
dtype: float64

Назначим минимальную зарплату всем, у кого ее нет (в данном случае только Сергею Бабочке): 

In [17]:
salaries[salaries.isnull()] = 1
salaries

Андрей             80000.0
Владимир           65000.0
Чарльз             20000.0
Анна               75000.0
Кот               300000.0
Сергей Бабочка         1.0
dtype: float64

В дальнейшем мы рассмотрим другие способы обработки пропусков в данных.

### Dataframe

Создадим pd.DataFrame из единичной numpy-матрицы:

In [32]:
df1 = pd.DataFrame(np.eye(3), index=['a', 'b', 'c'], 
                   columns=['col1', 'col2', 'col3'])
df1

,col1,col2,col3
a,1.0,0.0,0.0
b,0.0,1.0,0.0
c,0.0,0.0,1.0


Можно создавать pd.DataFrame из словаря.  
Ключами будут названия столбцов, а значениями - списки значений в этих столбцах.  
pd.DataFrame может хранить значения любых типов. Но в пределах одного столбца тип может быть только один:  

In [33]:
dictionary = {
    'A': np.arange(5),
    'B': ['a', 'b', 'c', 'd', 'e'],
    'C': np.arange(5) > 2
}
df2 = pd.DataFrame(dictionary)
df2

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,d,True
4,4,e,True


Можем обращаться к отдельному элементу в таблице через `at` (это быстро):

In [34]:
df2.at[3, 'B']

'd'

Можем обращаться к куску таблицы через loc (это всего лишь в [22 раза медленнее](https://stackoverflow.com/questions/37216485/pandas-at-versus-loc), чем at):

In [35]:
df2.loc[3:4, ['A', 'B']]

,A,B
3,3,d
4,4,e


Обращение только к строке:

In [36]:
# Нумерация начинается с 0
df2.loc[2]

A        2
B        c
C    False
Name: 2, dtype: object

Обращение только к столбцу.
Если мы хотим выделить все элементы по какой-то координате, можно написать просто `':'`:

In [37]:
# Здесь мы хотим взять все строки и пишем для этого ':'
df2.loc[:, 'B']

0    a
1    b
2    c
3    d
4    e
Name: B, dtype: object

Можем изменять элементы, обращаясь к ним через `at` и присваивая значение:

In [38]:
df2.at[3, 'B'] = 'Z'
df2

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True


С помощью loc можно изменять сразу всю строку.  
И даже создавать новые, смотрите:

In [39]:
df2.loc[5] = [77, '!', False]
df2

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True
5,77,!,False


Создадим копию нашей таблицы без последнего столбца.  
Затем, присоединим новую таблицу к старой и посмотрим, что будет.

In [42]:
df3 = df2.copy().loc[:, ['A', 'B']]
df3 = df2.append(df3)
df3

,A,B,C
0,0,a,False
1,1,b,False
2,2,c,False
3,3,Z,True
4,4,e,True
5,77,!,False
0,0,a,NaN
1,1,b,NaN
2,2,c,NaN
3,3,Z,NaN
